In [18]:
import os
import sys
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn import preprocessing
from sklearn.preprocessing import label_binarize
from nltk.stem import PorterStemmer
# import pyodbc
import collections
from sklearn.tree import DecisionTreeRegressor
DT = DecisionTreeRegressor()

In [19]:
data = pd.read_excel("Abeer Supreme Medical Center 2 ## 6212001674 @ 1285.xlsx")
data1 = data.copy()
len(data1)
data1.head()


,PROVIDER_CODE,SEGMENT_CODE,PROVIDER_NAME,SERVICE_CODE,SERVICE_NAME,SERVICE_TYPE
0,799102B,6212001674,Abeer Supreme Medical Center 2,1559-ENT,ARYTENOIDECTOMY,Package Deal
1,799102B,6212001674,Abeer Supreme Medical Center 2,1555-ENT,ORO ANTRAL FISTULA,Package Deal
2,799102B,6212001674,Abeer Supreme Medical Center 2,1556-ENT,TYMPANOPLASTY,Package Deal
3,799102B,6212001674,Abeer Supreme Medical Center 2,1557-ENT,STAPEDECTOMY,Package Deal
4,799102B,6212001674,Abeer Supreme Medical Center 2,1558-ENT,OPERATION ON VOCAL CORDS,Package Deal


In [7]:
create_document(data,3)

,Code_id,Block,Block Desc,ascii_desc,ascii_short_desc,effective_from,inactive
0,10801-00,1837,Speech audiometry,investigation evaluation fitting contact lenses,Investgtn evaln/fitting contact lenses,1998-07-01,2000-07-01
1,10816-00,1837,Speech audiometry,refitting contact lenses,Refitting of contact lenses,1998-07-01,2000-07-01
2,11000-00,1825,Electroencephalography [EEG],electroencephalography,Electroencephalography,1998-07-01,NaT
3,11003-00,1825,Electroencephalography [EEG],electroencephalography 3 hours duration,Electroencephalography of >= 3 hours,1998-07-01,NaT
4,11006-00,1825,Electroencephalography [EEG],temporosphenoidal electroencephalography,Temporosphenoidal electroencephalography,1998-07-01,NaT
...,...,...,...,...,...,...,...
7882,97981-00,490,Miscellaneous dental services,splinting stabilisation tooth direct,"Splint & stabilisation of tooth, direct",1998-07-01,NaT
7883,97982-00,490,Miscellaneous dental services,enamel stripping tooth,Enamel stripping of tooth,1998-07-01,NaT
7884,97983-00,490,Miscellaneous dental services,intravenous cannulation establishment infusion...,IV cannulation & est infusion by dentist,1998-07-01,2000-07-01
7885,97985-00,490,Miscellaneous dental services,provision oral appliance diagnosed snoring obs...,Prov oral appliance for snoring & OSA,2000-07-01,NaT


In [8]:
print(len(data1[data1['SERVICE_NAME'].str.contains('DAY PKG')]))
print(len(data1[data1['SERVICE_NAME'].str.contains('DAYS')]))
#print(len(ref1[ref1['LongDescription'].str.contains('CONSULTATION')]))
print(len(data1[data1['SERVICE_NAME'].isna()]))
#print(len(ref1[ref1['LongDescription'].isna()]))
#ref[ref['LongDescription'].str.contains('Scan')]
#ref[ref['LongDescription'].str.contains('CT')]
#ref[ref['LongDescription'].str.contains('C.T')]

KeyError: 'SERVICE_NAME'

In [ ]:
data1['SERVICE_NAME'] = data1['SERVICE_NAME'].str.replace('DAY PKG', '')
data1['SERVICE_NAME'] = data1['SERVICE_NAME'].str.replace('DAYS', '')
data1.head()

In [ ]:
tf1 = TfidfVectorizer()
data1['SERVICE_NAME'] = data1['SERVICE_NAME'].apply(lambda x: ''.join(i for i in x if not i.isdigit()))
tf1.fit_transform(data1['SERVICE_NAME'])
print(len(tf1.get_feature_names()))
data_features = pd.DataFrame(tf1.get_feature_names(), columns = ['Features'])
data_features.head()

In [ ]:
service_name = pd.DataFrame(data1['SERVICE_NAME'])

In [ ]:
service_name.head()

In [ ]:
ACHI = pd.read_excel('ACHI.XLSX')

In [ ]:
ACHI.head()

In [14]:
def create_document(DF1,col_index): #DF1 is the DF to be preprocessed #col_index is the number of the describtion column (int)
    stopWords = set(stopwords.words('english'))
    uselessWords = ['intended','for','use','helps','to','provide','all','detection','of']
    # Making A copy to prevent change to the original DF as it is needd subsequently
    DF = DF1.copy()
    # Insuring string type for the wanted col
    DF[DF.columns[col_index]] = DF[DF.columns[col_index]].astype(str)
    # Removing all newline and replacing it with space
    #DF[DF.columns[col_index]] = DF[DF.columns[col_index]].str.replace('\n',' ')
    #Replacing all non alphanumeric in both english and arabic by space
    #DF[DF.columns[col_index]] = DF[DF.columns[col_index]].apply(lambda x: re.sub('[^A-Za-z0-9]+', ' ', x))
    #Separating camel case with space
    #DF[DF.columns[col_index]] = DF[DF.columns[col_index]].apply(lambda x: re.sub(r"([a-z])([A-Z])", r"\1 \2", x))
    #lowercasing all describtions
    DF[DF.columns[col_index]] = DF[DF.columns[col_index]].apply(lambda x:" ".join(x.lower() for x in x.split()))
    #removing stop words
    DF[DF.columns[col_index]] = DF[DF.columns[col_index]].apply(lambda x:" ".join (word for word in x.split() if word not in stopWords))
    #removing useless words
    DF[DF.columns[col_index]] = DF[DF.columns[col_index]].apply(lambda x:" ".join(word for word in x.split() if word not in uselessWords))
    #DF[DF.columns[col_index]] = DF[DF.columns[col_index]].apply(lambda x:''.join(i for i in x if not i.isdigit()))
    #In case a med dict present (referring the medical abbreviation to its origin) this line return the abbreviation to origin
    ## DF_document.document = DF_document.document.apply(lambda x: " ".join(med_dict(word) for word in x.split()) )
    #Counting all Words that occur either in very low freq or very high and removing them (Threshhold must be configured)
    ## freq = pd.Series(' '.join(DF[DF.columns[col_index]]).split()).value_counts()[:50]
    ## DF[DF.columns[col_index]] = DF[DF.columns[col_index]].apply(lambda x: " ".join(x for x in x.split() if  x not in freq.index))
    ## freq = pd.Series(' '.join(DF[DF.columns[col_index]]).split()).value_counts()[-1034:]
    ## DF[DF.columns[col_index]] = DF[DF.columns[col_index]].apply(lambda x: " ".join(x for x in x.split() if x not in freq.index))
    #Defining a stemmer and stemming all words in the description
    #st = PorterStemmer()
    #DF[DF.columns[col_index]] = DF[DF.columns[col_index]].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
    #DF[DF.columns[col_index]] = DF[DF.columns[col_index]].apply(lambda x: "" if x.isdigit() == True else x)
    return DF
# create_document(data,4).to_excel('haha.xlsx')

In [ ]:
#Defining a function that take a single row string as first argument and the DF to be looked upon 
#this function return the index of service which has an exact match in the lookup or Null if there is not one
def dup_row_refrence(sd_desc,unique_list):
    for index,value in unique_list.items():
        if sd_desc == value:
            return index
    return 'NaN'

In [ ]:
#A function that uses dup_row_refrence by taking 1 DF and refrencing it by its self
def dup_dup(DF_dup, col_name):
    #Dropping duplicate from DF to create a refrence (drop duplicate except First)
    DF_unique =  DF_dup[col_name].drop_duplicates()
    #getting the subset of the services of processed DF that occur in refrence (all except first also)
    DF_dup = DF_dup[DF_dup.duplicated(subset = col_name)]
    #applying dup_row_refrence for each row
    dup_ref = DF_dup[col_name].apply(lambda x: dup_row_refrence(x,DF_unique))
    return dup_ref


In [ ]:
# A function that uses dup_row_refrence by taking 2 DF and refrencing one by the other
def dup_ref(DF_ref,DF_dup):
    #Dropping duplicate from refrence
    DF_unique =  DF_ref["ascii_desc"].drop_duplicates()
    #getting the subset of the services of processed DF that occur in refrence
    DF_dup = DF_dup[DF_dup["SERVICE_NAME"].isin(DF_unique)]
    #applying dup_row_refrence for each row
    dup_ref = DF_dup['SERVICE_NAME'].apply(lambda x: dup_row_refrence(x,DF_unique))
    return dup_ref

In [ ]:
ACHI_processed = create_document(ACHI, 3)

In [ ]:
ACHI_processed[ACHI_processed.columns[3]]

In [ ]:
print(len(ACHI_processed))
ACHI_processed['ascii_desc'].nunique()

In [ ]:
tfobj = TfidfVectorizer()
tf = tfobj.fit_transform(ACHI_processed['ascii_desc'])
ACHI_features = pd.DataFrame(tfobj.get_feature_names(), columns=['Features'])
ACHI_features.head()

In [ ]:
len(data_features[data_features['Features'].isin(ACHI_features['Features']) == True])

In [ ]:
# A function that uses dup_row_refrence by taking 2 DF and refrencing one by the other
def dup_ref_ACHI(DF_ref,DF_dup):
    #Dropping duplicate from refrence
    DF_unique =  DF_ref["ascii_desc"].drop_duplicates()
    #getting the subset of the services of processed DF that occur in refrence
    DF_dup = DF_dup[DF_dup["SERVICE_NAME"].isin(DF_unique)]
    #applying dup_row_refrence for each row
    dup_ref = DF_dup['SERVICE_NAME'].apply(lambda x: dup_row_refrence(x,DF_unique))
    return dup_ref

In [ ]:
data_processed = create_document(data1,4)

In [ ]:
dup_ref_ACHI(ACHI_processed, data_processed)

In [ ]:
ACHI_processed_ = dup_dup(ACHI_processed, 'ascii_desc')
print('len of duplicated services --> {}'.format(len(ACHI_processed_)))
print(len(collections.Counter(ACHI_processed['ascii_desc'])))
ACHI_processed_

In [ ]:
DF_data = pd.DataFrame()
DF_ref = pd.DataFrame()
DF_data['Name'] = data_processed['SERVICE_NAME']
DF_ref['Name'] = ACHI_processed['ascii_desc']
DF = pd.concat(objs = [DF_data,DF_ref])
DF['Name'] = DF['Name'].apply(lambda x: ''.join(i for i in x if not i.isdigit()))
len(DF)

In [ ]:
stopWords = stopwords.words('english')
tfidf = TfidfVectorizer(stop_words=stopWords)

In [ ]:
tfidf.fit(DF['Name'])
tfref = tfidf.transform(ACHI_processed['ascii_desc'])
tfdata = tfidf.transform(data_processed['SERVICE_NAME'])
print(tfdata.shape)
print(tfref.shape)

In [ ]:
#This function finds the (rank) most similar description from 1 tfidf matrixin another and return it
def find_similar(tfidf_data,tfidf_ref, index, rank = 0):
    #multiplying the 2 matrix to find the cosine similarity
    cosine_similarities = linear_kernel(tfidf_data[index:index+1], tfidf_ref[:]).flatten()
    #getting the most similar index
    highest_ind = cosine_similarities.argsort()[::-1][rank]
    #return the index along with its similarity
    return highest_ind, cosine_similarities[highest_ind]

In [ ]:
%%time
result = pd.DataFrame(columns = ['Serv','cpt','Sim','Num'])
resultAll = pd.DataFrame(columns = ['Serv','cpt','Sim','Num'])
for i in range(len(data_processed)):
    for j in range(0,3):
        index, simlarity = find_similar(tfdata, tfref, i, j)
        result.loc[i] = [i, index, simlarity, j]
        resultAll = resultAll.append(result)
        result.drop(result.index, inplace = True)

In [ ]:
#data.iloc[1,5]

In [ ]:
len(resultAll[resultAll['Sim'] > 0.5])

In [ ]:
result = resultAll.copy()
result['Serv'] = result['Serv'].astype(int)
result['cpt'] = result['cpt'].astype(int)
result['Num'] = result['Num'].astype(int)
result.head()

In [ ]:
first_pred = pd.DataFrame(columns=['Serv','cpt','Sim','Num'])
second_pred = pd.DataFrame(columns=['Serv','cpt','Sim','Num'])
third_pred = pd.DataFrame(columns=['Serv','cpt','Sim','Num'])

first_pred = result[result['Num'] == 0]
second_pred = result[result['Num'] == 1]
third_pred = result[result['Num'] == 2]

In [ ]:
first_pred.columns = ['Serv', 'ACHI_1','Sim_1','Num']
second_pred.columns = ['Serv', 'ACHI_2','Sim_2','Num']
third_pred.columns = ['Serv', 'ACHI_3','Sim_3','Num']

In [ ]:
AllResult = pd.concat(objs = [first_pred,second_pred, third_pred], axis=1, sort=False)
AllResult.head()

In [ ]:
AllResult.drop(['Num'], axis=1, inplace=True)
AllResult.columns = ['Al Rajhi Service','ACHI_1','Sim_1','Serv','ACHI_2','Sim_2','Serv','ACHI_3','Sim_3']
AllResult.drop(['Serv'], axis=1, inplace=True)
AllResult.head()

In [ ]:
data.head()

In [ ]:
AllResult.head()

In [ ]:
AllResult = AllResult.set_index('Al Rajhi Service')
AllResult = pd.concat(objs = [AllResult, data[['PROVIDER_CODE','SEGMENT_CODE','PROVIDER_NAME','SERVICE_CODE','SERVICE_NAME','SERVICE_TYPE']]],axis = 1 , join_axes=[AllResult.index] )
AllResult.head()

In [ ]:
AllResult = AllResult.reset_index()
AllResult = AllResult.set_index('ACHI_1')
AllResult = pd.concat(objs = [AllResult, ACHI[['ascii_short_desc','Code_id','Block']]], axis = 1, join_axes=[AllResult.index])
AllResult = AllResult.reset_index()
AllResult.head()

In [ ]:
AllResult.drop(['Al Rajhi Service'], axis=1, inplace=True)
AllResult.head()

In [ ]:
AllResult.drop(['ACHI_1'],axis =1 ,inplace=True)
AllResult.head()

In [ ]:
AllResult.columns = ['Score_1','ACHI_2','Score_2','ACHI_3','Score_3','PROVIDER_CODE','SEGMENT_CODE','PROVIDER_NAME','SERVICE_CODE','SERVICE_NAME','SERVICE_TYPE','ACHI_1','Code_id_1','Block_1']
AllResult.head()

In [ ]:
AllResult['SCORE'] = AllResult['Score_1']
AllResult.drop(['Score_1'], axis=1, inplace=True)
AllResult.columns = ['ACHI_2','Score_2','ACHI_3','Score_3','PROVIDER_CODE','SEGMENT_CODE','PROVIDER_NAME','SERVICE_CODE','SERVICE_NAME','SERVICE_TYPE','ACHI_1','MBS_1','Block_1','Score_1']
AllResult.head()

In [ ]:
AllResult = AllResult.set_index('ACHI_2')
AllResult = pd.concat(objs = [AllResult,ACHI[['ascii_desc','Code_id','Block']] ], axis = 1, join_axes=[AllResult.index])
AllResult.head(2)

In [ ]:
AllResult = AllResult.reset_index()
AllResult['SCORE'] = AllResult['Score_2']
AllResult.drop(['ACHI_2','Score_2'], axis=1, inplace=True)
AllResult.columns = ['ACHI_3','Score_3','PROVIDER_CODE','SEGMENT_CODE','PROVIDER_NAME','SERVICE_CODE','SERVICE_NAME','SERVICE_TYPE','ACHI_1','MBS_1','Block_1','Score_1','ACHI_2','Code_id_2','Block_2','Score_2']
AllResult.head(2)

In [ ]:
AllResult = AllResult.set_index('ACHI_3')
AllResult = pd.concat(objs = [AllResult, ACHI[['ascii_desc','Code_id','Block']] ], axis = 1, join_axes=[AllResult.index])
AllResult.head(2)

In [ ]:
AllResult = AllResult.reset_index()
AllResult['SCORE'] = AllResult['Score_3']
AllResult.drop(['ACHI_3','Score_3'], axis=1, inplace=True)
AllResult.columns = ['PROVIDER_CODE','SEGMENT_CODE','PROVIDER_NAME','SERVICE_CODE','SERVICE_NAME','SERVICE_TYPE','ACHI_1','Code_id_1','Block_1','Score_1','ACHI_2','Code_id_2','Block_2','Score_2','ACHI_3','Code_id_3','Block_3','Score_3']
AllResult.head(2)

In [ ]:
AllResult = AllResult.sort_values(by = ['Score_1','Score_2','Score_3'], ascending=False)

In [ ]:
len(AllResult[AllResult.Score_1 > 0.5])/len(AllResult)

In [ ]:
AllResult.head()

In [ ]:
def parse(DF,col_index):
    DF[DF.columns[col_index]] = DF[DF.columns[col_index]].astype(str)
    DF[DF.columns[col_index]] = DF[DF.columns[col_index]].apply(lambda x: x[3:] if x[:3] =='- -' else x)
    return DF

In [ ]:
AllResult_processed = parse(AllResult,3)
AllResult_processed = parse(AllResult_processed,7)
AllResult_processed = parse(AllResult_processed,11)

In [ ]:
AllResult_processed.head(20)

In [ ]:
AllResult_processed.to_csv('result.csv')

In [ ]:
# def get_index(x):
#     return service_name['Service Name'][service_name['Service Name'] == str(x)].index[0]

In [ ]:
# def get_similar(l1,l2,description):
#     l3 = []
#     for i in l1:
#         l3.append(description.values[np.argmax([cosine_similarity(i,j) for j in l2])][0])
#     return l3    

In [ ]:
# def find_similar(tfidf_data,tfidf_ref, index, rank = 1):
#     #multiplying the 2 matrix to find the cosine similarity
#     cosine_similarities = linear_kernel(tfidf_data[index:index+1], tfidf_ref[:]).flatten()
#     #getting the most similar index
#     highest_ind = cosine_similarities.argsort()[::-1][rank]
#     #return the index along with its similarity
#     return highest_ind, cosine_similarities[highest_ind]

In [ ]:
# service_name.iloc[1][0]

In [ ]:
# result = pd.DataFrame(columns = ['Serv','achi','Sim'])
# resultAll = pd.DataFrame(columns = ['Serv','achi','Sim'])
# for i in range(len(service_name)):
#     index, simlarity = find_similar(sn_vect, desc_vectorized, i, 1)
#     result.loc[i] = [service_name.iloc[i][0], achi.iloc[index][1], simlarity]
#     resultAll = resultAll.append(result)
#     result.drop(result.index, inplace = True)

In [ ]:
# resultAll.columns = ['Service Name', 'Achi','Similarity']

In [ ]:
# resultAll.head()

In [ ]:
#get_similar(sn_vect,desc_vectorized,description)

In [ ]:
#data['ACHI'].apply(lambda x : description.iloc[np.argmax([cosine_similarity(sn_vect[get_index(x)],j) for j in desc_vectorized]),0]).to_csv(r'D:\\Users\\Testing\\Desktop\\al-rajhai\\result.csv')